In [4]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

# lower limit of tumor volume = 1mm^3

# set up input box

lambda1 = 0.192 #[1/day]
b = 5.85 #[1/day]
d = 0.00873 #[1/day*mm^2]
e = 0.66 #[1/day*mg]
#u(t) = #angiogenic inhibitor effect
#tstep = #yo mama
epi = 0.000001
max_error = 10
guess_net = 105
guess_vas = 15
sigfigs = 4
icount = 0 
n0 = 100 #initial tumor volume in mm^3
v0 = 10    #initial vascular volume in mm^3 

umax = 13 #mg/kg of body mass of the drug 

net = []
vas = []
days = []
while n0 > 10: #while loop to solve for volume 
    icount += 1 
    if icount < 65 :
        umax = 13 
    else : 
        umax = 8.5+4.5**(-(icount-65))
    dd1 = -lambda1*n0*math.log10(n0/v0) #net tumor volume equation #derivitive of net volume function
    dd2 = b*n0-d*(n0^(2/3)*v0-e*v0*umax #derivitive of vascular volume function
    dt = 1 
    n1 = n0 + dt*dd1 #establish new height 
    v1 = v0 + dt*dd2 
    n0 = n1
    v0 = v1
    
    days.append('%.*g'%(icount) #add to time list 
    net.append('%.*g'%(sigfigs-1,n1)) #add to net volume list 
    vas.append('%.*g'%(sigfigs-1,v1)) #add to vas volume list 
print(net)
print(vas)
print(days)

SyntaxError: invalid syntax (1255044060.py, line 38)